In [1]:
!pip install aiostream ipywidgets
%env USER cms-jovyan

from coffea import hist, processor
from coffea.processor.servicex import DataSource, Analysis
from coffea.processor.servicex import DaskExecutor, LocalExecutor 
from func_adl import ObjectStream

import awkward as ak
import pandas as pd
import servicex as sx
from func_adl_servicex import ServiceXSourceUpROOT
from func_adl_uproot import UprootDataset
from servicex import ServiceXDataset

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
env: USER=cms-jovyan


In [2]:
class Processor(Analysis):
    @staticmethod
    def process(events):
        from collections import defaultdict
        
        sumw = defaultdict(float)
        output = hist.Hist(
            "Events",
            hist.Cat("dataset", "Dataset"),
            hist.Bin("jet_pt", "$Jet p_T", 100, 15, 60),
        )
        
        dataset = events.metadata["dataset"]
        jet_pt = events.Jet.pt
        print(jet_pt)
        
        # Fill the histogram
        sumw[dataset] += len(events)
        print('test1')
        print(len(events))
        print('test2')
        print(dataset)
        print('test3')
        print(MET)
        print('test4')
        output.fill(
            dataset=dataset,
            MET=ak.flatten(MET),
        )
        
        return {
            "sumw": sumw,
            "Jet_pt": output
        }

In [3]:
def get_JetpT(source: ObjectStream) -> ObjectStream:
    return source.Select(lambda event: {'Jet_pt': event.Jet_pt})

ds = ServiceXSourceUpROOT("root://eospublic.cern.ch//eos/root-eos/benchmark/Run2012B_SingleMu.root", 'Events', backend_name='uproot')
ds.return_qastle = True

dataset = [ServiceXDataset("root://eospublic.cern.ch//eos/root-eos/benchmark/Run2012B_SingleMu.root", backend_name='uproot')]
datasource = DataSource(query=get_JetpT(ds), metadata={'dataset': 'Run2012B_SingleMu'}, datasets=dataset)

In [4]:
from dask.distributed import Client

analysis = Processor()
executor = LocalExecutor()
#executor = DaskExecutor(client_addr="tls://matousadamec-40gmail-2ecom.dask.coffea.casa:8786")

async def run_updates_stream(accumulator_stream):
  global first
  count = 0
  async for coffea_info in accumulator_stream:
    count += 1
    print(count, coffea_info)
  return coffea_info
output = await run_updates_stream(executor.execute(analysis, datasource))

ValueError: offsets too short for ListOffsetArray: expected 53446198, offsets length - 1 is 1048576

(https://github.com/scikit-hep/awkward-1.0/blob/1.5.0/src/awkward/operations/convert.py#L4711)

In [ ]:
hist.plot1d(output['MET_pt'], overlay='dataset', fill_opts={'edgecolor': (0,0,0,0.3), 'alpha': 0.8})